Read all available news from publico.pt.
To choose only tech news change urlPublico to the following
'''python
def urlPublico(year, month, day):
    return "https://arquivo.pt/wayback/cdx?url=publico.pt/{year:04d}/{month:02d}/{day:02d}/tecnologia&matchType=prefix&output=json".format(year = year, month = month, day = day)
'''
To get news from different dates change start_date and/or end_date

In [ ]:
import requests, datetime, time, json

def urlPublico(year, month, day):
    return "https://arquivo.pt/wayback/cdx?url=publico.pt/{year:04d}/{month:02d}/{day:02d}/&matchType=prefix&output=json".format(year = year, month = month, day = day)

output = []

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 1, 5)
date = start_date

while (date < end_date):
    if (date.day == 1):
        print(date)
    r = requests.get(url = urlPublico(date.year, date.month, date.day))
    if (r.status_code == 200 and r.text != ""):
        for line in r.text.split('\n'):
            if (line != ""):
                output.append(json.loads(line))
    elif (r.status_code == 429):
        print("Reached threshold. Waiting 10 seconds")
        time.sleep(10)
    date = date + datetime.timedelta(days = 1)

results = {"publico" : output}

In [ ]:
# print number of news on a site
for identifier, result in results.items():
    print("site: {0}\t noticias: {1}".format(identifier, len(result)))

Clear the news that were fetched.
Multiple news with different timestamps are cleared.
Some websites that are not news are cleared.
Count the number of indexations for each of the news.

In [ ]:
# Clear repeated news
news = {}
number_of_indexations = {}

for identifier, result in results.items():
    for link in result:
        splitted_url = link['url'].split('/')
        if (len(splitted_url) < 9): # not news
            continue
        tittle = splitted_url[8]
        if ("." in tittle): # not news
            continue
        if (news.get(tittle) == None):
            news[tittle] = link
            number_of_indexations[tittle] = 1
        else:
            number_of_indexations[tittle] += 1


In [ ]:
print(len(news))

Fetch the website for the links obtained.
Some sites need to be cleaned (they are not news).

In [ ]:
websites = {}

def create_website_link(url, timestamp):
    return "https://arquivo.pt/noFrame/replay/{0}/{1}".format(timestamp, url)

for tittle, link in news.items():
    response = requests.get(create_website_link(link['url'], link['timestamp']))
    websites[tittle] = response.text

# len between 372500 and 37599 might be 404 page


In the end we have 3 dictionaries with the same keys (tittle of the news).
News has all the info provided in the initial API requests.
Number_of_indexations has the number of different timestamps for each website.
Websites contains the HTML for each of the websites.

In [ ]:
print(len(websites))
print(news['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])
print(number_of_indexations['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])
print(websites['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])